# LORA Fine-tuning

In [1]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    TFAutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import tensorflow
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

In [ ]:

dataset = load_dataset('deepset/prompt-injections')
dataset

### model

In [ ]:
model_checkpoint = "distilbert-base-uncased"
model_output = "distilbert-base-uncased-deepset-promptinjection"

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, problem_type="single_label_classification")

In [ ]:
# display architecture
model

### preprocess data

In [5]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


In [7]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [8]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

### Train model

In [9]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [ ]:
peft_config

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [12]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 5

In [13]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

In [14]:
# show metrics for f1 and auprc
from sklearn.metrics import f1_score, precision_recall_curve, auc

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision, recall, _ = precision_recall_curve(labels, preds)
    auprc = auc(recall, precision)
    return {
    'f1': f1,
    'auprc': auprc
    }

In [15]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [ ]:
# train model
trainer.train()

In [17]:
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # you can name the model whatever you want

In [18]:
model.push_to_hub(model_id) # save model

In [19]:
trainer.push_to_hub(model_output) # save trainer